# Updating Data in SQLite Tables

In this notebook, you'll learn how to update existing data in SQLite tables using UPDATE statements. We'll cover basic updates, conditional updates, and best practices.

In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect('university.db')
cursor = conn.cursor()

print('Connected to database')

## Basic UPDATE Syntax

The basic syntax for UPDATE is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```

Always use WHERE clause to avoid updating all rows!

In [ ]:
# First, let's see current student data
cursor.execute('SELECT id, name, grade FROM students')
students_before = cursor.fetchall()

print('Students before update:')
for student in students_before:
    print(f'ID: {student[0]}, Name: {student[1]}, Grade: {student[2]}')

# Update a specific student's grade
cursor.execute('''
    UPDATE students
    SET grade = 3.9
    WHERE id = 1
''')

print('\nUpdated student with ID 1')

## Updating Multiple Columns

You can update multiple columns in a single UPDATE statement.

In [ ]:
# Update multiple columns for a student
cursor.execute('''
    UPDATE students
    SET age = 23, grade = 3.7
    WHERE name = 'Bob Smith'
''')

print('Updated Bob Smith\'s age and grade')

# Check the update
cursor.execute('SELECT name, age, grade FROM students WHERE name = "Bob Smith"')
bob = cursor.fetchone()
print(f'Bob Smith: Age {bob[1]}, Grade {bob[2]}')

## Conditional Updates

Use WHERE clause with various conditions to update specific rows.

In [ ]:
# Update all students with grade below 3.0 to 3.0
cursor.execute('''
    UPDATE students
    SET grade = 3.0
    WHERE grade < 3.0
''')

print('Updated low-performing students to minimum grade of 3.0')

# Update students in certain age range
cursor.execute('''
    UPDATE students
    SET grade = grade + 0.1
    WHERE age BETWEEN 20 AND 22
''')

print('Gave grade bonus to students aged 20-22')

## Using Parameters in Updates

Always use parameters for user input to prevent SQL injection.

In [ ]:
# Update using parameters
student_id = 2
new_grade = 4.0
new_age = 21

cursor.execute('''
    UPDATE students
    SET grade = ?, age = ?
    WHERE id = ?
''', (new_grade, new_age, student_id))

print(f'Updated student ID {student_id} with parameterized query')

# Using named parameters
cursor.execute('''
    UPDATE courses
    SET credits = :credits
    WHERE course_name = :course_name
''', {
    'credits': 3.5,
    'course_name': 'Introduction to Computer Science'
})

print('Updated course credits using named parameters')

## Updating with Subqueries

You can use subqueries in UPDATE statements.

In [ ]:
# Update department budget based on number of courses
cursor.execute('''
    UPDATE departments
    SET budget = budget * 1.1
    WHERE dept_id IN (
        SELECT DISTINCT d.dept_id
        FROM departments d
        JOIN courses c ON d.dept_name = c.department
        GROUP BY d.dept_id
        HAVING COUNT(c.course_id) > 1
    )
''')

print('Increased budget for departments with multiple courses')

## Checking Update Results

Verify that updates were applied correctly.

In [ ]:
# Check updated students
cursor.execute('SELECT id, name, age, grade FROM students ORDER BY id')
students_after = cursor.fetchall()

print('Students after updates:')
for student in students_after:
    print(f'ID: {student[0]}, Name: {student[1]}, Age: {student[2]}, Grade: {student[3]}')

# Check affected rows
print(f'\nRows affected by last update: {cursor.rowcount}')

# Check courses
cursor.execute('SELECT course_name, credits FROM courses')
courses = cursor.fetchall()

print('\nUpdated courses:')
for course in courses:
    print(f'{course[0]}: {course[1]} credits')

## UPDATE with JOIN

SQLite supports UPDATE with JOIN (using subqueries since it doesn't support direct JOIN in UPDATE).

In [ ]:
# Update enrollment grades based on student performance
cursor.execute('''
    UPDATE enrollments
    SET grade = 'A'
    WHERE student_id IN (
        SELECT id FROM students WHERE grade >= 3.8
    )
''')

print('Updated enrollment grades for high-performing students')

## Best Practices for Updates

- Always use WHERE clause to avoid updating all rows
- Use parameters for user input
- Test updates on a copy of data first
- Use transactions for multiple related updates
- Backup data before major updates

In [ ]:
# Example of safe update with transaction
try:
    conn.execute('BEGIN TRANSACTION')
    
    # Update within transaction
    cursor.execute('''
        UPDATE students
        SET grade = grade + 0.1
        WHERE grade < 4.0
    ''')
    
    # Check if update affected expected number of rows
    if cursor.rowcount > 0:
        conn.commit()
        print('Transaction committed - grades updated')
    else:
        conn.rollback()
        print('Transaction rolled back - no rows affected')
        
except sqlite3.Error as e:
    conn.rollback()
    print(f'Error during update: {e}')

# Commit all changes
conn.commit()

## Summary

In this notebook, you learned how to:

- Perform basic UPDATE operations
- Update multiple columns simultaneously
- Use WHERE clauses for conditional updates
- Use parameterized queries for security
- Update with subqueries
- Verify update results
- Use transactions for safe updates

Remember to always backup your data before performing updates, and use WHERE clauses to avoid accidentally updating all rows in a table.

In [ ]:
# Close the connection
conn.close()
print('Database connection closed')